In [243]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from skimage.color import rgb2gray, rgba2rgb
import math

sector_size = 9
idx = 5


In [244]:
def toGrayScale(img):
    imgGray = img
    if(img.shape[2] == 4):
        imgGray = rgb2gray(rgba2rgb(img))
        return imgGray
    imgGray = rgb2gray(img)
    return imgGray

def applyFFT(img):
    imgFFT = np.fft.fft2(img)
    return imgFFT

def applyIFFT(img):
    imgIFFT = np.fft.ifft2(img)
    return imgIFFT

def encode(img1, img2):
    imgBase = applyFFT(img1)
    height, width = imgBase.shape
    
    imgSecret = applyFFT(img2)
    imgSecretFlat = imgSecret.ravel()
    ind = 0
    
    if((height*width)/sector_size**2 < len(imgSecretFlat)):
        print('Encoding not possible')
    
    for x in range (0, height-idx, sector_size):
        for y in range (0, width-idx, sector_size):
            if ind < len(imgSecretFlat):
                imgBase[x+idx, y+idx] = imgSecretFlat[ind]
                #print(ind,x+idx,y+idx)
                ind += 1
            else:
                break
    
    return applyIFFT(imgBase)

def decode(img1,img2):
    imgBase = applyFFT(img1)
    height, width = imgBase.shape
    
    secret_h, secret_w = img2.shape
    secret = np.zeros((secret_h*secret_w), dtype=np.complex128)
    ind = 0
    for x in range (0, height-idx, sector_size):
        for y in range (0, width-idx, sector_size):
            if ind < len(secret):
                secret[ind] = imgBase[x+idx, y+idx]
                #print(ind,x+idx,y+idx)
                ind += 1
            else:
                break
    secret = secret.reshape((secret_h,secret_w))
    return applyIFFT(secret)
    

In [245]:
imgBaseName = './images/input.jpg'
imgSecretName = './images/qr.png'

imgBase = plt.imread(imgBaseName)

imgBaseGray = toGrayScale(imgBase)
matplotlib.image.imsave('./images/pre.png', imgBaseGray)

imgSecret = plt.imread(imgSecretName)
imgSecretGray = toGrayScale(imgSecret)

In [ ]:
fig, ax = plt.subplots(1,4,figsize=(15,15))
ax[0].set_title('Base Image', fontsize = 15)
ax[0].imshow(imgBaseGray, cmap='gray')

ax[1].set_title('Secret Message', fontsize = 15)
ax[1].imshow(imgSecretGray, cmap='gray')

finalImage = encode(imgBaseGray, imgSecretGray)
matplotlib.image.imsave('./images/encoded.png', abs(finalImage))
ax[2].set_title('Encoded Image', fontsize = 15)
ax[2].imshow(abs(finalImage), cmap='gray')

secret = decode(finalImage,imgSecretGray)
matplotlib.image.imsave('./images/decoded.png', abs(secret))
ax[3].set_title('Decoded Message', fontsize = 15)
ax[3].imshow(abs(secret), cmap='gray')